In [16]:
# Make sure to run this cell before any other
# Import necessary modules
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import requests
import time
import random
from datetime import datetime
from datetime import timedelta
from pprint import pprint
import json
import isodate

import googleapiclient.discovery
from googleapiclient.errors import HttpError
from config import api_key

# Define API Key (Replace with your API key)
API_KEY = api_key
max_results = 50

# Initialize YouTube API client
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

In [4]:
# Function to use Youtube API and find all channels related to 'query'
def fetch_all_channels(query, max_results=50):
    next_page_token = None
    all_channels = []

    while True:
        request = youtube.search().list(
            part="snippet",
            q=query,
            type="channel",
            maxResults=max_results,
            order="date",
            pageToken=next_page_token 
        )
        response = request.execute()

        all_channels.extend(response.get("items", []))
        
        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return all_channels

In [5]:
# Use function above to find all channels related to DIY
channels = fetch_all_channels("DIY")

In [ ]:
# Print total amount of channels found
print(f"Total channels retrieved: {len(channels)}")

In [9]:
# Function to use Youtube API and find all channel data related to channelId's we got previously
def fetch_all_channels_data(channel_list_id, max_results=50):
    all_channels_data = []
    batch_size = 50  # Max allowed per API call
    
    # Grouping channel IDs into batches of 50
    for i in range(0, len(channel_list_id), batch_size):
        batch_ids = [channel_info['id']['channelId'] for channel_info in channel_list_id[i:i + batch_size]]
        
        # Fetch channel data in batches of 50
        request = youtube.channels().list(
            part="snippet,statistics,contentDetails,brandingSettings,topicDetails,status",
            id=",".join(batch_ids)  # Multiple channel IDs separated by commas
        )
        response = request.execute()

        all_channels_data.extend(response.get("items", []))

    return all_channels_data

In [10]:
# Use function above to find all channel data related to channelId's found
channels_data = fetch_all_channels_data(channels)

In [ ]:
# Print total amount of channels found
print(f"Total channels retrieved: {len(channels_data)}")

In [15]:
# Write the json data pulled for channels to a file
filename = 'data/channels.json'
with open(filename, 'w') as file:
    json.dump(channels, file, indent=4)

In [16]:
# Write the json data pulled for channels_data to a file
filename = 'data/channels_data.json'
with open(filename, 'w') as file:
    json.dump(channels_data, file, indent=4)

In [27]:
# START HERE AFTER RUNNING FIRST CELL IF YOU DONT WANT TO USE API CALLS ON CHANNELS

# If you didnt run the API calls, run this to read in prevous json data from channels.json
filename = 'data/channels.json'
with open(filename, 'r') as file:
    channels = json.load(file)

In [28]:
# If you didnt run the API calls, run this to read in prevous json data from channels_data.json
filename = 'data/channels_data.json'
with open(filename, 'r') as file:
    channels_data = json.load(file)

In [29]:
# This code block will take the json file, grab the needed data, and turn it into a dataframe

channel_list = [] # List to hold channel data dictionaries
for channel in channels_data:
    channel_title = channel['brandingSettings']['channel']['title'] # get channel title
    channel_id = channel['id']
    try:
        channel_subCount = channel['statistics']['subscriberCount'] # get channel subscriber count, if it fails ...
    except KeyError:
        channel_subCount = '???' # set subscriber count to ???
    channel_videoCount = channel['statistics']['videoCount']
    channel_viewCount = channel['statistics']['viewCount']
    channel_upload_playlist = channel["contentDetails"]["relatedPlaylists"]["uploads"]

    # Build a dictionary for the channel and add it to the list
    channel_list.append({'channel_title':channel_title,
                            'channel_id':channel_id,
                            'channel_upload_playlist':channel_upload_playlist,
                            'channel_subCount':channel_subCount,
                            'channel_videoCount':channel_videoCount,
                            'channel_viewCount':channel_viewCount})

# Create a DF from list
channel_data_df = pd.DataFrame(channel_list)

In [30]:
# Check the dtypes of columns
channel_data_df.dtypes

channel_title              object
channel_id                 object
channel_upload_playlist    object
channel_subCount           object
channel_videoCount         object
channel_viewCount          object
dtype: object

In [31]:
# Change some columns to int64
channel_data_df['channel_subCount'] = channel_data_df['channel_subCount'].astype('int64')
channel_data_df['channel_videoCount'] = channel_data_df['channel_videoCount'].astype('int64')
channel_data_df['channel_viewCount'] = channel_data_df['channel_viewCount'].astype('int64')

In [32]:
# Display the sorted dataframe based on subcount
channel_data_df.sort_values('channel_subCount',ascending=False).head(20)

,channel_title,channel_id,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
479,5-Minute Crafts,UC295-Dw_tDNtZXFeAPAW6Aw,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
430,5-Minute Crafts DIY,UC2etEuPIfohP4P53wM0KImA,UU2etEuPIfohP4P53wM0KImA,20300000,6640,5187110540
438,"SLICK SLIME SAM - DIY, Comedy, Science",UCw5VDXH8up3pKUppIvcstNQ,UUw5VDXH8up3pKUppIvcstNQ,11200000,1477,5389464086
342,Ideas en 5 minutos DIY,UCxBRABv7DQJqNn6zwkVFGsA,UUxBRABv7DQJqNn6zwkVFGsA,6260000,6306,1223828792
334,DIY & Crafts,UCwToG9qbHnaAFrllqt9TmNw,UUwToG9qbHnaAFrllqt9TmNw,5100000,19,1231019252
372,Ideias Incríveis DIY,UCxFRrC-Xup5LVd7tyt0AGAQ,UUxFRrC-Xup5LVd7tyt0AGAQ,4680000,6606,960715967
432,５分でできる DIY,UCJsxAriYBUnKZVdMXZnGDoA,UUJsxAriYBUnKZVdMXZnGDoA,3860000,6189,1666614796
505,Home RenoVision DIY,UCnorhjQR4zJkT7AVNhu395Q,UUnorhjQR4zJkT7AVNhu395Q,3410000,941,559273237
551,DIY Creators,UChKlSK39lLg8eZHIX0iVzLA,UUhKlSK39lLg8eZHIX0iVzLA,3220000,200,253390531
466,DIY Queen,UCSFKV-1TQsNpDvOv8H6UAQA,UUSFKV-1TQsNpDvOv8H6UAQA,3160000,329,674190084


In [33]:
# Show the total amount of videos among every channel in channel_data_df
print(sum(channel_data_df['channel_videoCount'].values))

270166


In [ ]:
# This cell will pull every uploaded video id and published date for every channel in channel_data_df

video_ids = []
all_video_data = []

MAX_RETRIES = 3

for index, row in channel_data_df.iterrows():
    channelid = row['channel_id']
    uploads_playlist_id = row['channel_upload_playlist']

    print(f"📌 Pulling data for (ChannelID: {channelid}, PlaylistID: {uploads_playlist_id}), Total Videos Sofar: {len(video_ids)} ...")
    
    next_page_token = None
    retries = 0

    while True:
        try:
            request = youtube.playlistItems().list(
                part="snippet,contentDetails,status",
                playlistId=uploads_playlist_id,
                maxResults=50,  # Max per request
                pageToken=next_page_token
            )
            response = request.execute()

            # Handle empty playlists (avoid KeyError)
            items = response.get("items", [])
            if not items:
                print(f"⚠️ No videos found for ChannelID: {channelid}")
                break  # Stop fetching if the playlist is empty

            for item in items:
                video_ids.append({
                    'channel_id': channelid,
                    'video_id': item["snippet"]["resourceId"]["videoId"],
                    'publishedAt': item["snippet"]['publishedAt']
                })

            all_video_data.extend(items)

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break  # No more pages

        except HttpError as e:
            print(f"❌ API Error for Channel {channelid}: {e}")
            if e.resp.status == 403:  # Quota exceeded or rate limit
                print("⚠️ Quota exceeded or rate limited. Sleeping for 10 minutes...")
                time.sleep(600)  # Sleep 10 minutes before retrying
            elif e.resp.status in [500, 503]:  # Server errors (retryable)
                print("⚠️ Server error, retrying in 5 seconds...")
                time.sleep(5)
            else:
                print("⚠️ Non-retryable error, skipping this channel.")
                break  # Stop retrying on fatal errors
            
            retries += 1
            if retries >= MAX_RETRIES:
                print(f"⚠️ Too many failures for Channel {channelid}, skipping...")
                break  # Avoid infinite loops on repeated errors
    
    time.sleep(random.uniform(0.5, 1.5))

# Convert to DataFrame and Save as CSV
video_id_df = pd.DataFrame(video_ids)
csv_output_path = "data/youtube_videos_list.csv"
video_id_df.to_csv(csv_output_path, index=False)
print(f"📂 CSV file saved: {csv_output_path}")

# Save the raw json data incase needed
json_filename = 'data/video_id_data.json'
try:
    with open(json_filename, 'w', encoding='utf-8') as file:
        json.dump(all_video_data, file, ensure_ascii=False, indent=4)
    print(f"📂 JSON file saved: {json_filename}")
except Exception as e:
    print(f"❌ Error saving JSON file: {e}")

print(f"✅ Data collection complete. Total videos collected: {len(video_id_df)}")

In [49]:
video_id_df.head(20)

,channel_id,video_id,publishedAt
0,UCjAzObYQ1--u0ctr2Kt_PuQ,myYfDsxcDA0,2025-01-17T08:41:29Z
1,UCjAzObYQ1--u0ctr2Kt_PuQ,TKhDNl5JqxM,2025-01-17T08:36:14Z
2,UCjAzObYQ1--u0ctr2Kt_PuQ,eww1W2QEX18,2025-01-15T19:34:59Z
3,UCjAzObYQ1--u0ctr2Kt_PuQ,5s1d6JQqfIU,2025-01-15T19:32:48Z
4,UCjAzObYQ1--u0ctr2Kt_PuQ,LgTwITNhWqk,2025-01-15T19:29:23Z
5,UCSvJyHffQ48dw2oQVq9cgqQ,IQYwsJQOecU,2025-01-17T23:15:00Z
6,UCSvJyHffQ48dw2oQVq9cgqQ,BgFHhNl9U7k,2025-01-15T11:33:56Z
7,UCcAA1NgTk2TnKdwHEEflTWQ,u-wFPHOAY0E,2025-01-17T19:56:13Z
8,UCEPzmkd5mqbwA6zIeVvFz4w,ykfSwWzM-SM,2025-01-18T12:18:11Z
9,UCEPzmkd5mqbwA6zIeVvFz4w,_p3O5mbHsx4,2025-01-18T09:01:01Z


In [54]:
# Convert to datetime
videoid_clean_time_df = video_id_df.copy()
videoid_clean_time_df['publishedAt'] = pd.to_datetime(video_id_df['publishedAt'])

# Extract date and time
videoid_clean_time_df['date_publishedAt'] = videoid_clean_time_df['publishedAt'].dt.date
videoid_clean_time_df['time_publishedAt'] = videoid_clean_time_df['publishedAt'].dt.time
videoid_clean_time_df = videoid_clean_time_df.drop(columns='publishedAt')
videoid_clean_time_df

,channel_id,video_id,date_publishedAt,time_publishedAt
0,UCjAzObYQ1--u0ctr2Kt_PuQ,myYfDsxcDA0,2025-01-17,08:41:29
1,UCjAzObYQ1--u0ctr2Kt_PuQ,TKhDNl5JqxM,2025-01-17,08:36:14
2,UCjAzObYQ1--u0ctr2Kt_PuQ,eww1W2QEX18,2025-01-15,19:34:59
3,UCjAzObYQ1--u0ctr2Kt_PuQ,5s1d6JQqfIU,2025-01-15,19:32:48
4,UCjAzObYQ1--u0ctr2Kt_PuQ,LgTwITNhWqk,2025-01-15,19:29:23
...,...,...,...,...
268966,UC_rUFviaLzBXUNcks0j2l4g,RUUhDi8tBTU,2015-02-05,17:04:07
268967,UC_rUFviaLzBXUNcks0j2l4g,tGSzBe8jrcE,2015-02-03,19:30:23
268968,UC_rUFviaLzBXUNcks0j2l4g,sJLNUwW6_bg,2015-02-02,22:40:09
268969,UC_rUFviaLzBXUNcks0j2l4g,aE3ArY9BBww,2015-01-30,15:45:50


In [59]:
# Merging both the channel_data_df and videoid_clean_time_df
channel_video_merged_df = pd.merge(videoid_clean_time_df, channel_data_df, on='channel_id', how='left')
print(channel_video_merged_df.shape) 
channel_video_merged_df.head(20)

(268971, 9)


,channel_id,video_id,date_publishedAt,time_publishedAt,channel_title,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
0,UCjAzObYQ1--u0ctr2Kt_PuQ,myYfDsxcDA0,2025-01-17,08:41:29,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
1,UCjAzObYQ1--u0ctr2Kt_PuQ,TKhDNl5JqxM,2025-01-17,08:36:14,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
2,UCjAzObYQ1--u0ctr2Kt_PuQ,eww1W2QEX18,2025-01-15,19:34:59,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
3,UCjAzObYQ1--u0ctr2Kt_PuQ,5s1d6JQqfIU,2025-01-15,19:32:48,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
4,UCjAzObYQ1--u0ctr2Kt_PuQ,LgTwITNhWqk,2025-01-15,19:29:23,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
5,UCSvJyHffQ48dw2oQVq9cgqQ,IQYwsJQOecU,2025-01-17,23:15:00,ENA Diy,UUSvJyHffQ48dw2oQVq9cgqQ,1,1,2
6,UCSvJyHffQ48dw2oQVq9cgqQ,BgFHhNl9U7k,2025-01-15,11:33:56,ENA Diy,UUSvJyHffQ48dw2oQVq9cgqQ,1,1,2
7,UCcAA1NgTk2TnKdwHEEflTWQ,u-wFPHOAY0E,2025-01-17,19:56:13,Fan DIY,UUcAA1NgTk2TnKdwHEEflTWQ,0,1,0
8,UCEPzmkd5mqbwA6zIeVvFz4w,ykfSwWzM-SM,2025-01-18,12:18:11,DiY Arts and craft,UUEPzmkd5mqbwA6zIeVvFz4w,0,3,0
9,UCEPzmkd5mqbwA6zIeVvFz4w,_p3O5mbHsx4,2025-01-18,09:01:01,DiY Arts and craft,UUEPzmkd5mqbwA6zIeVvFz4w,0,3,0


In [62]:
# Check for any NA values
print(channel_video_merged_df.isna().any())

channel_id                 False
video_id                   False
date_publishedAt           False
time_publishedAt           False
channel_title              False
channel_upload_playlist    False
channel_subCount           False
channel_videoCount         False
channel_viewCount          False
dtype: bool


In [63]:
# Save the merged dataframe to a CSV
file_path = "data/full_videos_list.csv"
channel_video_merged_df.to_csv(file_path, index=False)

In [26]:
# Read in the full_video_list.csv if you didnt run previous code
channel_video_merged_df = pd.read_csv("data/full_videos_list.csv")
channel_video_merged_df

,channel_id,video_id,date_publishedAt,time_publishedAt,channel_title,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
0,UCjAzObYQ1--u0ctr2Kt_PuQ,myYfDsxcDA0,2025-01-17,08:41:29,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
1,UCjAzObYQ1--u0ctr2Kt_PuQ,TKhDNl5JqxM,2025-01-17,08:36:14,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
2,UCjAzObYQ1--u0ctr2Kt_PuQ,eww1W2QEX18,2025-01-15,19:34:59,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
3,UCjAzObYQ1--u0ctr2Kt_PuQ,5s1d6JQqfIU,2025-01-15,19:32:48,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
4,UCjAzObYQ1--u0ctr2Kt_PuQ,LgTwITNhWqk,2025-01-15,19:29:23,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
...,...,...,...,...,...,...,...,...,...
268966,UC_rUFviaLzBXUNcks0j2l4g,RUUhDi8tBTU,2015-02-05,17:04:07,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
268967,UC_rUFviaLzBXUNcks0j2l4g,tGSzBe8jrcE,2015-02-03,19:30:23,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
268968,UC_rUFviaLzBXUNcks0j2l4g,sJLNUwW6_bg,2015-02-02,22:40:09,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
268969,UC_rUFviaLzBXUNcks0j2l4g,aE3ArY9BBww,2015-01-30,15:45:50,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576


In [27]:
# Change date published to a datetime dtype
channel_video_merged_df['date_publishedAt'] = pd.to_datetime(channel_video_merged_df['date_publishedAt'])
print(channel_video_merged_df.dtypes)
channel_video_merged_df

channel_id                         object
video_id                           object
date_publishedAt           datetime64[ns]
time_publishedAt                   object
channel_title                      object
channel_upload_playlist            object
channel_subCount                    int64
channel_videoCount                  int64
channel_viewCount                   int64
dtype: object


,channel_id,video_id,date_publishedAt,time_publishedAt,channel_title,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
0,UCjAzObYQ1--u0ctr2Kt_PuQ,myYfDsxcDA0,2025-01-17,08:41:29,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
1,UCjAzObYQ1--u0ctr2Kt_PuQ,TKhDNl5JqxM,2025-01-17,08:36:14,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
2,UCjAzObYQ1--u0ctr2Kt_PuQ,eww1W2QEX18,2025-01-15,19:34:59,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
3,UCjAzObYQ1--u0ctr2Kt_PuQ,5s1d6JQqfIU,2025-01-15,19:32:48,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
4,UCjAzObYQ1--u0ctr2Kt_PuQ,LgTwITNhWqk,2025-01-15,19:29:23,DIY or DIE,UUjAzObYQ1--u0ctr2Kt_PuQ,0,5,33
...,...,...,...,...,...,...,...,...,...
268966,UC_rUFviaLzBXUNcks0j2l4g,RUUhDi8tBTU,2015-02-05,17:04:07,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
268967,UC_rUFviaLzBXUNcks0j2l4g,tGSzBe8jrcE,2015-02-03,19:30:23,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
268968,UC_rUFviaLzBXUNcks0j2l4g,sJLNUwW6_bg,2015-02-02,22:40:09,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
268969,UC_rUFviaLzBXUNcks0j2l4g,aE3ArY9BBww,2015-01-30,15:45:50,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576


In [28]:
# Grab only videos published in 2024
channel_video_2024_df = channel_video_merged_df[channel_video_merged_df['date_publishedAt'].dt.year == 2024]
channel_video_2024_df

,channel_id,video_id,date_publishedAt,time_publishedAt,channel_title,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
133,UCMRnsnlHHtVt0WmTzoVyk-w,tQ5qWH05TvA,2024-12-31,12:36:22,DIY queen,UUMRnsnlHHtVt0WmTzoVyk-w,16,132,3047
134,UCMRnsnlHHtVt0WmTzoVyk-w,olOANmxDR30,2024-12-31,12:31:05,DIY queen,UUMRnsnlHHtVt0WmTzoVyk-w,16,132,3047
135,UCMRnsnlHHtVt0WmTzoVyk-w,9f5yz00-pc8,2024-12-30,12:14:28,DIY queen,UUMRnsnlHHtVt0WmTzoVyk-w,16,132,3047
136,UCMRnsnlHHtVt0WmTzoVyk-w,eTXdvWjtLY0,2024-12-29,11:40:14,DIY queen,UUMRnsnlHHtVt0WmTzoVyk-w,16,132,3047
137,UCMRnsnlHHtVt0WmTzoVyk-w,t2WWCMPvd1E,2024-12-29,11:25:44,DIY queen,UUMRnsnlHHtVt0WmTzoVyk-w,16,132,3047
...,...,...,...,...,...,...,...,...,...
266622,UC_rUFviaLzBXUNcks0j2l4g,1rS7DxsbAPQ,2024-01-16,17:22:28,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
266623,UC_rUFviaLzBXUNcks0j2l4g,BZ4k8zByVVI,2024-01-10,20:19:49,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
266624,UC_rUFviaLzBXUNcks0j2l4g,W4nr1HbrI1o,2024-01-10,20:17:01,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576
266625,UC_rUFviaLzBXUNcks0j2l4g,X5oDk3vnq4M,2024-01-10,20:16:09,DIY Life Tech,UU_rUFviaLzBXUNcks0j2l4g,15700,2587,13942576


In [29]:
# Save the videos from 2024 dataframe to a CSV
file_path = "data/videos_2024_list.csv"
channel_video_2024_df.to_csv(file_path, index=False)

In [ ]:
# Read in the videos_2024_list.csv if you did run previous code
channel_2024_videos_df = pd.read_csv("data/videos_2024_list.csv")
channel_2024_videos_df = channel_2024_videos_df.sort_values('channel_subCount',ascending=False)
channel_2024_videos_df

,channel_id,video_id,date_publishedAt,time_publishedAt,channel_title,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
35374,UC295-Dw_tDNtZXFeAPAW6Aw,juYonFfdxJQ,2024-09-10,13:00:53,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35390,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,2024-09-02,11:00:48,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35397,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,2024-08-30,11:00:54,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35396,UC295-Dw_tDNtZXFeAPAW6Aw,EGzGak2IyrA,2024-08-30,13:00:13,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35395,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,2024-08-31,11:00:26,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
...,...,...,...,...,...,...,...,...,...
719,UCx2grh-o99YnDXcy0Pf7clg,vSDsYrWQUio,2024-12-05,04:53:38,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120
717,UCx2grh-o99YnDXcy0Pf7clg,3i2Gxbuf8v4,2024-12-09,11:20:11,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120
716,UCx2grh-o99YnDXcy0Pf7clg,X-Rj8IGrACo,2024-12-15,21:43:42,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120
725,UCx2grh-o99YnDXcy0Pf7clg,aJjO1LGtG6U,2024-11-30,14:15:58,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120


In [ ]:
# This cell will pull all video data for every channel in channel_data_df

all_video_data = []
batch_size = 50
MAX_RETRIES = 3
starting_point = 0  # <---------------------CHANGE THIS FOR YOUR STARTING POINT
last_processed_index = starting_point  # Track last completed batch
quota_limit_reached = False

for i in range(starting_point, len(channel_2024_videos_df), batch_size):
    retries = 0
    batch = channel_2024_videos_df['video_id'][i:i + batch_size].astype(str).tolist()

    while (retries < MAX_RETRIES) and not quota_limit_reached:
        print(f"📌 Pulling data for Total Videos So Far: {len(all_video_data)} ...")

        try:
            request = youtube.videos().list(
                part="snippet,statistics,contentDetails,status,topicDetails,player,liveStreamingDetails",
                id=",".join(batch),
            )
            response = request.execute()

            # Extract video data from response
            items = response.get("items", [])
            if not items:
                print(f"⚠️ No videos found for Batch {i // batch_size + 1}")
            else:
                all_video_data.extend(items)

            last_processed_index = i
            break

        except HttpError as e:
            print(f"❌ API Error for Batch {i // batch_size + 1}: {e}")
            
            if e.resp.status == 403:  # Quota exceeded or rate limit
                print(f"⚠️ Quota exceeded at index {i}. Stopping...")
                quota_limit_reached = True
                last_processed_index = i
                break
            
            elif e.resp.status in [500, 503]:  # Retryable server errors
                print("⚠️ Server error, retrying in 5 seconds...")
                time.sleep(5)
            else:
                print("⚠️ Non-retryable error, skipping this batch.")
                break
            
            retries += 1
            if retries >= MAX_RETRIES:
                print(f"⚠️ Too many failures for Batch {i // batch_size + 1}, skipping...")
                break

    if quota_limit_reached:  # Stop processing if quota is reached
        print(f"🚨 Quota limit reached! Stopping at index {i}")
        break

    # Randomized sleep to prevent hitting rate limits
    time.sleep(random.uniform(0.5, 1.5))

# ---------------------Save last processed index for resumption----------------------
index_filename = 'data/last_index.txt'
try:
    with open(index_filename, 'w', encoding='utf-8') as file:
        file.write(str(last_processed_index))
    print(f"📌 Last processed index saved: {last_processed_index}")
except Exception as e:
    print(f"❌ Error saving last index file: {e}")
# -----------------------------------------------------------------------------------

# -------------------------Save results to JSON file---------------------------------
json_filename = f"data/videoData2024_{last_processed_index}.json"
try:
    with open(json_filename, 'w', encoding='utf-8') as file:
        json.dump(all_video_data, file, ensure_ascii=False, indent=4)
    print(f"📂 JSON file saved: {json_filename}")
except Exception as e:
    print(f"❌ Error saving JSON file: {e}")
# -----------------------------------------------------------------------------------

print(f"✅ Data collection complete. Total videos collected: {len(all_video_data)}")
print(f"🔄 Resume from index: {last_processed_index}")


In [32]:
# Read in the json file if you did run the code above
file_path = "data/videoData2024_45350.json"
with open(file_path, "r", encoding="utf-8") as file:
    read_in_data = json.load(file)

In [33]:
# This code block will use the json data to build a dataframe
list_of_videos = []

for item in read_in_data:
    try:
        # Collect data for each item
        video_data_row = {
            "c_channel_id": item['snippet']['channelId'],
            "v_video_id": item['id'],
            "v_category_id": item['snippet'].get('categoryId', 'N/A'),  # Use default if missing
            "v_tags": item['snippet'].get('tags', []),
            "v_title": item['snippet'].get('title', 'N/A'),
            "v_description": item['snippet'].get('description', 'N/A'),
            "v_comment_count": item['statistics'].get('commentCount', '0'),
            "v_likes": item['statistics'].get('likeCount', '0'),
            "v_favorites": item['statistics'].get('favoriteCount', '0'),
            "v_views": item['statistics'].get('viewCount', '0'),
            "v_definition": item['contentDetails'].get('definition', 'N/A'),
            "v_duration": item['contentDetails'].get('duration', 'N/A'),
            "v_caption": item['contentDetails'].get('caption', 'N/A'),
            "v_age_restricted": item['contentDetails'].get('contentRating', {}).get('ytRating', 'Not Restricted'),
            "v_privacy_status": item['status'].get('privacyStatus', 'N/A'),
            "v_made_for_kids": item['status'].get('madeForKids', False)
        }
        list_of_videos.append(video_data_row)
    except KeyError as e:
        print(f"Missing key: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

full_video_data_df = pd.DataFrame(list_of_videos)
full_video_data_df

,c_channel_id,v_video_id,v_category_id,v_tags,v_title,v_description,v_comment_count,v_likes,v_favorites,v_views,v_definition,v_duration,v_caption,v_age_restricted,v_privacy_status,v_made_for_kids
0,UC295-Dw_tDNtZXFeAPAW6Aw,juYonFfdxJQ,26,"[5 minute craft, 5 minutes craft, 5-minute cra...",LEVEL UP YOUR BARBIE DOLL WITH THESE STUNNING ...,LEVEL UP YOUR BARBIE DOLL with these STUNNING ...,23,491,0,62211,hd,PT1H39S,false,Not Restricted,public,False
1,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,26,"[5 minute craft, 5 minutes craft, 5-minute cra...",BRILLIANT LIFE HACKS 🌟💙 COOL DOLL REUSE,Discover brilliant life hacks and creative way...,28,442,0,76602,hd,PT2H3M35S,false,Not Restricted,public,False
2,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,26,"[5 minute craft, 5 minutes craft, 5-minute cra...",SECRET SMART COOKING HACKS THAT CHANGE EVERYTH...,🍳 Ready to elevate your cooking game? In this ...,21,1165,0,341133,hd,PT2H1M48S,false,Not Restricted,public,False
3,UC295-Dw_tDNtZXFeAPAW6Aw,EGzGak2IyrA,26,"[5 minute craft, 5 minutes craft, 5-minute cra...",CAN’T DEAL WITH STRESS? TRY THESE SATISFYING D...,🌟 Feeling stressed? Discover satisfying hacks ...,27,344,0,65712,hd,PT1H39S,false,Not Restricted,public,False
4,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,26,"[5 minute craft, 5 minutes craft, 5-minute cra...",INCREDIBLE RAINBOW IDEAS 🌈 Creative Room Makeo...,🎨 Ready to transform your space into something...,43,1374,0,222425,hd,PT1H4S,false,Not Restricted,public,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45384,UCx2grh-o99YnDXcy0Pf7clg,vSDsYrWQUio,22,[],موديل تنبات في كمام المبتدئات #تنبات #الراندة...,,0,0,0,1,hd,PT4M17S,false,Not Restricted,public,False
45385,UCx2grh-o99YnDXcy0Pf7clg,3i2Gxbuf8v4,22,[],تنبات وردة بالموزون#1#الراندة_المغربية #اكسبلو...,,0,2,0,3,hd,PT16M12S,false,Not Restricted,public,False
45386,UCx2grh-o99YnDXcy0Pf7clg,X-Rj8IGrACo,22,[],اسماء الوان الاثواب بالفرنسية#الراندة_المغربية...,,0,1,0,3,hd,PT1M23S,false,Not Restricted,public,False
45387,UCx2grh-o99YnDXcy0Pf7clg,aJjO1LGtG6U,22,[],فتح نصف طوق المبتدئات #الراندة_المغربية,,0,1,0,3,hd,PT9M26S,false,Not Restricted,public,False


In [34]:
# Rename channel data video frame columns to match with new data columns
# channel_id,video_id,date_publishedAt,time_publishedAt,channel_title,channel_upload_playlist,channel_subCount,channel_videoCount,channel_viewCount
channel_2024_videos_df = channel_2024_videos_df.rename(columns={
                                                                "channel_id": "c_channel_id",
                                                                "video_id": "v_video_id",
                                                                "date_publishedAt":"v_date_publishedAt",
                                                                "time_publishedAt":"v_time_publishedAt",
                                                                "channel_title":"c_channel_title",
                                                                "channel_upload_playlist":"c_channel_upload_playlist",
                                                                "channel_subCount":"c_channel_subCount",
                                                                "channel_videoCount":"c_channel_videoCount",
                                                                "channel_viewCount":"c_channel_viewCount"
                                                                })

channel_2024_videos_df

,c_channel_id,v_video_id,v_date_publishedAt,v_time_publishedAt,c_channel_title,c_channel_upload_playlist,c_channel_subCount,c_channel_videoCount,c_channel_viewCount
35374,UC295-Dw_tDNtZXFeAPAW6Aw,juYonFfdxJQ,2024-09-10,13:00:53,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35390,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,2024-09-02,11:00:48,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35397,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,2024-08-30,11:00:54,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35396,UC295-Dw_tDNtZXFeAPAW6Aw,EGzGak2IyrA,2024-08-30,13:00:13,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
35395,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,2024-08-31,11:00:26,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663
...,...,...,...,...,...,...,...,...,...
719,UCx2grh-o99YnDXcy0Pf7clg,vSDsYrWQUio,2024-12-05,04:53:38,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120
717,UCx2grh-o99YnDXcy0Pf7clg,3i2Gxbuf8v4,2024-12-09,11:20:11,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120
716,UCx2grh-o99YnDXcy0Pf7clg,X-Rj8IGrACo,2024-12-15,21:43:42,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120
725,UCx2grh-o99YnDXcy0Pf7clg,aJjO1LGtG6U,2024-11-30,14:15:58,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120


In [35]:
# Merge Channel data with video data making sure they match on both channel id and video id
full_data_merged_df = channel_2024_videos_df.merge(
    full_video_data_df,
    on=["c_channel_id", "v_video_id"],
    how="inner"
)
full_data_merged_df

,c_channel_id,v_video_id,v_date_publishedAt,v_time_publishedAt,c_channel_title,c_channel_upload_playlist,c_channel_subCount,c_channel_videoCount,c_channel_viewCount,v_category_id,...,v_comment_count,v_likes,v_favorites,v_views,v_definition,v_duration,v_caption,v_age_restricted,v_privacy_status,v_made_for_kids
0,UC295-Dw_tDNtZXFeAPAW6Aw,juYonFfdxJQ,2024-09-10,13:00:53,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,23,491,0,62211,hd,PT1H39S,false,Not Restricted,public,False
1,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,2024-09-02,11:00:48,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,28,442,0,76602,hd,PT2H3M35S,false,Not Restricted,public,False
2,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,2024-08-30,11:00:54,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,21,1165,0,341133,hd,PT2H1M48S,false,Not Restricted,public,False
3,UC295-Dw_tDNtZXFeAPAW6Aw,EGzGak2IyrA,2024-08-30,13:00:13,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,27,344,0,65712,hd,PT1H39S,false,Not Restricted,public,False
4,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,2024-08-31,11:00:26,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,43,1374,0,222425,hd,PT1H4S,false,Not Restricted,public,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45384,UCx2grh-o99YnDXcy0Pf7clg,vSDsYrWQUio,2024-12-05,04:53:38,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,0,0,1,hd,PT4M17S,false,Not Restricted,public,False
45385,UCx2grh-o99YnDXcy0Pf7clg,3i2Gxbuf8v4,2024-12-09,11:20:11,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,2,0,3,hd,PT16M12S,false,Not Restricted,public,False
45386,UCx2grh-o99YnDXcy0Pf7clg,X-Rj8IGrACo,2024-12-15,21:43:42,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,1,0,3,hd,PT1M23S,false,Not Restricted,public,False
45387,UCx2grh-o99YnDXcy0Pf7clg,aJjO1LGtG6U,2024-11-30,14:15:58,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,1,0,3,hd,PT9M26S,false,Not Restricted,public,False


In [36]:
# Check for any NA values
full_data_merged_df = full_data_merged_df.replace("N/A", None)
print(full_data_merged_df.isna().any())

c_channel_id                 False
v_video_id                   False
v_date_publishedAt           False
v_time_publishedAt           False
c_channel_title              False
c_channel_upload_playlist    False
c_channel_subCount           False
c_channel_videoCount         False
c_channel_viewCount          False
v_category_id                False
v_tags                       False
v_title                      False
v_description                False
v_comment_count              False
v_likes                      False
v_favorites                  False
v_views                      False
v_definition                 False
v_duration                    True
v_caption                    False
v_age_restricted             False
v_privacy_status             False
v_made_for_kids              False
dtype: bool


In [37]:
# Drop all rows that have a blank
full_data_merged_df = full_data_merged_df.dropna()
print(full_data_merged_df.isna().any())
print(len(full_data_merged_df))

c_channel_id                 False
v_video_id                   False
v_date_publishedAt           False
v_time_publishedAt           False
c_channel_title              False
c_channel_upload_playlist    False
c_channel_subCount           False
c_channel_videoCount         False
c_channel_viewCount          False
v_category_id                False
v_tags                       False
v_title                      False
v_description                False
v_comment_count              False
v_likes                      False
v_favorites                  False
v_views                      False
v_definition                 False
v_duration                   False
v_caption                    False
v_age_restricted             False
v_privacy_status             False
v_made_for_kids              False
dtype: bool
45387


In [38]:
# Function to convert ISO 8601 duration to HH:MM:SS
def convert_duration(duration_str):
    if pd.isna(duration_str):  # Handle missing values
        return None
    duration = isodate.parse_duration(duration_str)  # Parse duration
    return str(timedelta(seconds=duration.total_seconds()))  # Convert to HH:MM:SS

In [39]:
# Apply function to the 'v_duration' column
full_data_merged_df['v_duration'] = full_data_merged_df['v_duration'].apply(convert_duration)
full_data_merged_df

,c_channel_id,v_video_id,v_date_publishedAt,v_time_publishedAt,c_channel_title,c_channel_upload_playlist,c_channel_subCount,c_channel_videoCount,c_channel_viewCount,v_category_id,...,v_comment_count,v_likes,v_favorites,v_views,v_definition,v_duration,v_caption,v_age_restricted,v_privacy_status,v_made_for_kids
0,UC295-Dw_tDNtZXFeAPAW6Aw,juYonFfdxJQ,2024-09-10,13:00:53,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,23,491,0,62211,hd,1:00:39,false,Not Restricted,public,False
1,UC295-Dw_tDNtZXFeAPAW6Aw,oGga7hZypCs,2024-09-02,11:00:48,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,28,442,0,76602,hd,2:03:35,false,Not Restricted,public,False
2,UC295-Dw_tDNtZXFeAPAW6Aw,pkzIiXkRxB0,2024-08-30,11:00:54,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,21,1165,0,341133,hd,2:01:48,false,Not Restricted,public,False
3,UC295-Dw_tDNtZXFeAPAW6Aw,EGzGak2IyrA,2024-08-30,13:00:13,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,27,344,0,65712,hd,1:00:39,false,Not Restricted,public,False
4,UC295-Dw_tDNtZXFeAPAW6Aw,TUS8vZaHNds,2024-08-31,11:00:26,5-Minute Crafts,UU295-Dw_tDNtZXFeAPAW6Aw,81100000,7445,27960945663,26,...,43,1374,0,222425,hd,1:00:04,false,Not Restricted,public,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45384,UCx2grh-o99YnDXcy0Pf7clg,vSDsYrWQUio,2024-12-05,04:53:38,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,0,0,1,hd,0:04:17,false,Not Restricted,public,False
45385,UCx2grh-o99YnDXcy0Pf7clg,3i2Gxbuf8v4,2024-12-09,11:20:11,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,2,0,3,hd,0:16:12,false,Not Restricted,public,False
45386,UCx2grh-o99YnDXcy0Pf7clg,X-Rj8IGrACo,2024-12-15,21:43:42,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,1,0,3,hd,0:01:23,false,Not Restricted,public,False
45387,UCx2grh-o99YnDXcy0Pf7clg,aJjO1LGtG6U,2024-11-30,14:15:58,DIY by lamiae,UUx2grh-o99YnDXcy0Pf7clg,5,28,120,22,...,0,1,0,3,hd,0:09:26,false,Not Restricted,public,False


In [40]:
# save the final merged dataframe to a csv
csv_output_path = "data/final_video_data.csv"
full_data_merged_df.to_csv(csv_output_path, index=False)